# Notebook de benchmarking de la Ecuación de Poisson
[Documentación](#https://juliaci.github.io/BenchmarkTools.jl/stable/manual/)

In [1]:
using Statistics, CSV, Tables

In [2]:
include("FVM.jl")

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path.\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.")
└ @ Plots C:\Users\oscar\.julia\packages\Plots\nqFaB\src\backends.jl:545


Main.FVM

## Ecuación de Poisson 3D

### Construcción de la malla

In [3]:
function benchmark_mesh(vols::Int)
    volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces = FVM.uniform_grid(vols, 10)
    tags = FVM.init_tags(3, volumes, centers_and_boundaries)
    tags_b = FVM.init_tags_boundaries(3, centers_and_boundaries)

    FVM.tag_wall(tags, tags_b, [:W, :E, :T, :N, :B], 0, :D)
    FVM.tag_wall(tags, tags_b, :S, 100, :D)

    mesh = FVM.Mesh(volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces, tags, tags_b);
end

benchmark_mesh (generic function with 1 method)

### Haciendo operaciones del FVM

In [4]:
function Γ_constant(x::Array, y::Array, z::Array)
    Γ = 1000
    tensor_Γ = ones(length(x), length(y), length(z))
    tensor_Γ = Γ .* tensor_Γ
end

Γ_constant (generic function with 1 method)

In [17]:
function benchmark_set_boundary_conditions(mesh::FVM.Mesh)
    coeff = FVM.init_coefficients(mesh)
    print(Γ_constant)
    FVM.set_diffusion(coeff, Γ_constant);
    return coeff
end

benchmark_set_boundary_conditions (generic function with 1 method)

### Obteniendo soluciones

In [6]:
function benchmark_solutions(coeff::FVM.Coefficients_3d)
    equation_system = FVM.init_eq_system(coeff)
    solution = FVM.get_solution(equation_system);
end

benchmark_solutions (generic function with 1 method)

---

## Midiendo tiempo más formalmente

In [15]:
function iteratevolumes(volumes::Array{Int, 1}, times::Int)
    for volume ∈ volumes
        mesh = benchmark_mesh(volume)
        coefficients = benchmark_set_boundary_conditions(mesh)
        
        functions = [benchmark_mesh, benchmark_set_boundary_conditions, benchmark_solutions]
        #functions = [benchmark_mesh]
        args = [volume, mesh, coefficients]
        #args = [volume]
        
        for (f, arg) ∈ zip(functions, args)
            println("Comencé el de $volume volúmenes con la función $f")
            list_of_time_statistics = list_of_statistics(f, arg, volume, times)
            writetofile(f, list_of_time_statistics')
        end
        println("Terminé el de $volume volúmenes")
    end
end

iteratevolumes (generic function with 1 method)

In [8]:
function list_of_statistics(f, arg, volume::Int, times::Int)
    time_list, μ, σ = getstatistics(f, arg, times)
    time_statistics = [volume, volume^3, μ, σ]
    push!(time_statistics, time_list...)
    return time_statistics
end

list_of_statistics (generic function with 1 method)

In [9]:
function getstatistics(f, arg, times)
    time_list = []
    for _ ∈ 1:(times+1)
        t = measuretime(f, arg)
        push!(time_list, t)
    end
    times_without_compiling = time_list[2:end]
    μ = mean(times_without_compiling)
    σ = std(times_without_compiling)
    
    return time_list, μ, σ
end

getstatistics (generic function with 1 method)

In [10]:
function measuretime(f, arg)
    start_time = time()
    f(arg)
    finish_time = time()
    t = finish_time - start_time
    return t
end

measuretime (generic function with 1 method)

In [11]:
function writetofile(f, list)
    file_name = string(f)
    #CSV.write("../benchmarking/$(file_name).csv", Tables.table(list), delim = ',',append=true)
end

writetofile (generic function with 1 method)

In [18]:
volumes = [10,20]
times = 10
iteratevolumes(volumes, times)

Γ_constantComencé el de 10 volúmenes con la función benchmark_mesh


Comencé el de 10 volúmenes con la función benchmark_set_boundary_conditions
Γ_constantΓ_constantΓ_constant

Γ_constantΓ_constantΓ_constantΓ_constant

Γ_constantΓ_constantΓ_constantΓ_constantComencé el de 10 volúmenes con la función benchmark_solutions


Terminé el de 10 volúmenes

...

Γ_constant

Γ_constant

Γ_constant

Comencé el de 20 volúmenes con la función benchmark_solutions


Terminé el de 20 volúmenes


---